### BenchMark Spark TF-IDF

In [ ]:
import pyspark
sc = pyspark.SparkContext(master="local[1]",appName="myAppName")
import pymongo_spark
pymongo_spark.activate()

In [ ]:
import jieba
from pyspark.sql import HiveContext
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import time
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

#time_start = time.clock()
mStart=time.time()
mongo_rdd = sc.mongoRDD('mongodb://10.120.28.29:27017/newstest.newstest5')
mEnd=time.time()
print '讀取資料花了 {} s'.format(mEnd-mStart)

tStart=time.time()
content = mongo_rdd.map(lambda x: x['content'])
wordRDD = content.map(lambda x:tuple(jieba.cut(x)))
#tEnd = time.time() #記時(時間結束)
#print ' 花了 {} s'.format(tEnd-tStart)
tEnd = time.time() #記時(時間結束)
print 'jieba花了 {} s'.format(tEnd-tStart)

fStart =time.time()
#time_start_jieba = time.clock()
hashingTF = HashingTF()
tf = hashingTF.transform(wordRDD)
#tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)
fEnd =time.time()
print 'tfidf花了 {} s'.format(fEnd-fStart)
#time_end_jieba = time.clock()
#time_for_jieba= time_end_jieba-time_start_jieba
#print (time_for_jieba)

#time_step2 = time.clock()
#print "計算tf-idf的時間", time_step2-time_step1, "秒"
#time_step1 = time.clock()
#print "jieba的時間", time_step1-time_start, "秒"


### BenchMark Python TF-IDF

In [ ]:
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
import time
jieba.set_dictionary('/home/cloudera/Desktop/text_mining/dict.txt')             #切換至中文繁體字庫
jieba.load_userdict("/home/cloudera/Desktop/text_mining/dict_keyw_new.txt")     #加入自建詞庫
jieba.load_userdict("/home/cloudera/Desktop/text_mining/dict_cbdic.txt")

#預設就是自己
client = MongoClient('10.120.28.29',27017)
database = client['newstest']
collection =database['newstest5']

In [ ]:
title=[]
content =[]
#拿資料
mStart=time.time()
col = collection.find()
mEnd=time.time()
print '讀取資料花了 {} s'.format(mEnd-mStart)
tStart=time.time()
for post in col:
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    #print content
    title.append(post['title'])
tEnd = time.time() #記時(時間結束)
print 'jieba花了 {} s'.format(tEnd-tStart)
    #print title
#總文章數量
#print "文章數:",len(title)
#定義計算TF-IDF的函式
fStart=time.time()
def tfIdf(content):
    vectorizer = TfidfVectorizer()
    #print vectorizer
    #help(vectorizer)
    X = vectorizer.fit_transform(content)  # titile 放文本
    #print X
    weight = X.toarray()
    #print weight
    #print "特徵值數量:",len(vectorizer.get_feature_names())
    return weight,X,vectorizer
#把上面存好的切詞進行TF-IDF計算，並把結果存進 weight
weight,X,vectorizer = tfIdf(content)
class test(): #宣告類別
    
    @staticmethod #宣告成靜態方法, 靜態方法不需要設定self
    def getTopTfidf(news_id,n):
        indices = np.argsort(X.toarray()[news_id])[::-1] 
        #print indices
        features = vectorizer.get_feature_names()
        #print features
        #print title[news_id]
        top_n = n
        top_features = [features[i] for i in indices[:top_n]]
        return top_features,indices

    @staticmethod
    def getTopWeight(news_id,n):
        top_features,indices = test.getTopTfidf(news_id,n) #類別名稱.方法()
        a=0
        for i in top_features:
            #print i,weight[news_id][indices[a]]  # weight[2000] 裡面請填入跟上面一樣的文章編號
            a=a+1
test.getTopWeight(40,50)
fEnd =time.time()
print 'tfidf花了 {} s'.format(fEnd-fStart)